## Virtual screening Dataset

Here we run all other scoring functions on the Dataset2 - the virtual screening dataset

In [ ]:
import pandas as pd
import numpy as np
import pickle as pk

### Get the data

In [ ]:
vs_dataset = pd.read_csv("../Datasets/virtual_screening_dataset.csv")

In [ ]:

def reformat_path(path):
    prefix = "/home/anja/Documents/"
    new = "../../"
    if not prefix in path: return path
    res = path[len(prefix):]
    return new+res


vs_dataset["path"] = vs_dataset["path"].apply(reformat_path)
vs_dataset

### Get the features

In [ ]:
def pppene_to_array(tmp):
    tmp = tmp.replace("(", "")
    tmp = tmp.replace(")", "")
    tmp = tmp.strip("[]")
    tmp = tmp.replace(" ", "")
    tmp = tmp.replace("\n", ",")
    return np.fromstring(tmp, dtype=float, sep=", ").reshape(9,20)

vs_dataset_feat = pd.read_csv("../Featurization/rosettaPPPEnergies_vsData.csv")
vs_dataset_feat["energies"] = vs_dataset_feat["energies"].apply(pppene_to_array)

In [ ]:
vs_dataset_feat = vs_dataset_feat[["allele", "peptide", "binder", "ba", "energies", "total_energy"]]
vs_dataset_feat["path"] = vs_dataset["path"]
vs_dataset_feat

## Score with 3pHLA-score

In [ ]:
root_dir = "../Experiment1 - train ref2015-score, standard-pHLA-score and 3pHLA-score/"

def get_energies(X):
    ene = np.roll(X, 4, axis = 0)[:9,:19]
    ene = np.roll(ene, -4, axis = 0)
    ene = ene.reshape(9*19)
    return ene

def score_3pHLA(row):
    print(row)
    allele = row["allele"]
    feat = row["energies"]

    model_9n = root_dir+"/final_REGRmodels/"+allele+"ppp.pkl"

    model_9 = pk.load(open(model_9n, 'rb'))
    pred1 = model_9.predict([get_energies(feat)])[0]
    return pred1

alleles = vs_dataset_feat["allele"].unique()

vs_dataset_feat["3pHLA-score"] = vs_dataset_feat.apply(score_3pHLA, axis=1)

vs_dataset_feat

In [ ]:
vs_dataset["3pHLA-score"] = vs_dataset_feat["3pHLA-score"]
vs_dataset[["allele", "peptide", "ba", "binder", "path", "allele_type", "3pHLA-score"]]

In [ ]:
vs_dataset.to_csv("Results2_tmp_3phla.csv")

## Score with Vina, Vinardo, AutoDock

In [ ]:
import HLA_Arena as arena
# Initialize scoring
import pandas as pd
import seaborn as sns

def score_vvad(row, func):
    print(row)
    ene = arena.rescore_complex_simple_smina(row["path"], func)
    print(ene)
    return ene
    
#to change scoring function instead of "vina", place "vinardo" or "ad4_scoring"
funcs = ["ad4_scoring", "vina", "vinardo"]
#funcs = ["ad4_scoring"]

for f in funcs:
    print("scoring with: "+f)
    vs_dataset_feat[f+"-score"] = vs_dataset_feat.apply(lambda x: score_vvad(x, f), axis=1)

## Score with DOPE

In [ ]:
from modeller import *
from modeller.automodel import *
from modeller.scripts import complete_pdb

def score_dope(fpath):
    print("-----------------------------------------------")
    print("SCORING WITH DOPE")
    print(fpath)
    #score    
    env = environ()
    env.libs.topology.read(file='$(LIB)/top_heav.lib')
    env.libs.parameters.read(file='$(LIB)/par.lib')
    
    mdl = complete_pdb(env, fpath)
        
    atmsel_lig = selection(mdl.chains[2])
    atmsel_rec = selection(mdl.chains[0], mdl.chains[1])
    atmsel_full = selection(mdl.chains[0], mdl.chains[1], mdl.chains[2])
    try:
        score_lig = atmsel_lig.assess_dope()
        score_rec = atmsel_rec.assess_dope()
        score_full = atmsel_full.assess_dope()
    except:
        print("Failed on: "+fpath)
        return (0, 0)
    
    score_diff = score_full - (score_rec + score_lig) 
    
    return (score_full, score_diff)

vs_dataset_feat["dope-score"] = vs_dataset_feat.apply(lambda row: score_dope(row["path"]), axis=1)

## Score with FoldX

In [ ]:
%cd ..

In [ ]:
import subprocess

def find_pHLA(file_path):
    if not "rdf_mount" in file_path:
        return file_path[file_path.rfind("/")+1:file_path.find(".pdb")]
    else:
        prefix = "../../rdf_mount/decoymulticonf/confs/"
        suffix = "/full_system_confs"
        return file_path[file_path.find(prefix)+len(prefix):file_path.find(suffix)]
    

def score_FX(row):
    print("processing "+row["path"])
    #score    
    env = environ()
    env.libs.topology.read(file='$(LIB)/top_heav.lib')
    env.libs.parameters.read(file='$(LIB)/par.lib')
    
    
    mdl = complete_pdb(env, row["path"])
    chain_rec = mdl.chains[0].name
    chain_pep = mdl.chains[2].name
    
    cname = find_pHLA(row["path"])
    command = "./script_rescore_FoldX.sh "+chain_pep+" "+chain_rec+" "+row["path"]+" "+cname
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    process.wait()
    ret = process.returncode
    print(ret)
    if not ret == 0: print("Error with execution") 
    print("done with "+row["path"])
    
def reformat_path(path):
    old = "../../"
    new = "/data/"
    if not old in path: return path
    res = path[len(old):]
    return new+res

In [ ]:
process = subprocess.Popen("touch FoldX_results.csv", shell=True, stdout=subprocess.PIPE)
process = subprocess.Popen("echo \"name, ene\" > FoldX_results.csv", shell=True, stdout=subprocess.PIPE)
process.wait()
process.returncode

In [ ]:
vs_dataset_feat["path"] = vs_dataset_feat["path"].apply(reformat_path)
vs_dataset_feat.apply(lambda row: score_FX(row), axis=1) 

In [ ]:
res_tmp = pd.read_csv("FoldX_results.csv")
res_tmp["foldx-score"] = [" ene"].apply(lambda x: float(x))
vs_dataset_feat["foldx-score"] = res_tmp["foldx-score"]

## Score with GradDock

In [ ]:
%cd /graddock/GD/evaluate
%pwd

In [ ]:
import subprocess
## Access ligand/receptor files

def extract_receptor_ligand(filename):
    env = environ()
    env.libs.topology.read(file='$(LIB)/top_heav.lib')
    env.libs.parameters.read(file='$(LIB)/par.lib')
    mdl = complete_pdb(env, filename)
    
    atmsel_lig = selection(mdl.chains[2])
    atmsel_rec = selection(mdl.chains[0], mdl.chains[1])
    
    lig_name = filename[:filename.find(".pdb")]+"_ligand.pdb"
    rec_name = filename[:filename.find(".pdb")]+"_receptor.pdb"
    atmsel_lig.write(lig_name)
    atmsel_rec.write(rec_name)
    
    return (lig_name, rec_name)
    
def get_rec_lig_name(path):
    
    if "singleconf" in path:
        lname = lfname[lfname.rfind("/")+1:]
        rname = rfname[rfname.rfind("/")+1:]
        return ()
    
def score_GD(row):
    print("processing "+row["path"])
    (lfname, rfname) = extract_receptor_ligand(row["path"])
    folder_src = row["path"][:row["path"].rfind("/")]
    lname = lfname[lfname.rfind("/")+1:]
    rname = rfname[rfname.rfind("/")+1:]
    command = "./GD_run.sh "+lname+" "+rname+" "+folder_src+" > GD_progress.txt"
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    process.wait()
    command = "rm "+lfname
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    process.wait()
    command = "rm "+rfname
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    process.wait()
    ret = process.returncode
    print(ret)
    if not ret == 0: print("Error with execution") 
    print("done with "+row["path"])
    
def reformat_path(path):
    old = "../../"
    new = "/data/"
    if not old in path: return path
    res = path[len(old):]
    return new+res

In [ ]:
process = subprocess.Popen("touch GD_results.csv", shell=True, stdout=subprocess.PIPE)
process = subprocess.Popen("echo \"name, complex, diff\" GD_results.csv", shell=True, stdout=subprocess.PIPE)
process.wait()
process.returncode

In [ ]:
dataset["path"] = dataset["path"].apply(reformat_path)
vs_dataset_feat.apply(lambda row: score_GD(row), axis=1)

In [ ]:
res_tmp = pd.read_csv("GD_results.csv")
res_tmp["graddock-score"] = [" diff"].apply(lambda x: float(x))
vs_dataset_feat["graddock-score"] = res_tmp["graddock-score"]

In [ ]:
vs_dataset_feat.to_csv("Results2.csv")